In [ ]:
from copy import deepcopy
import os
import sys
from datetime import datetime

In [ ]:
import matplotlib.pyplot as plt
from numba import njit
import numpy as np
import pandas as pd

from scipy.optimize import basinhopping, brute, differential_evolution, dual_annealing, shgo
from scipy.stats import chisquare

sys.path.append(os.path.dirname(sys.path[0]))

In [ ]:
import scripts.constants as con
from scripts.helpers import get_unique_frame
from scripts.pca import PCA
from scripts.properties.ovito_procedures import OvitoProcessor
from scripts.rdf_calculator import RDFCalculator

In [ ]:
plt.style.use('seaborn')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['figure.figsize'] = [i / 2.54 for i in (15, 15)]
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['mathtext.it'] = 'Times New Roman'
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['axes.labelsize'] = 14

In [ ]:
np.random.seed(0)

In [ ]:
@njit
def math_round(value):
    rest = value - int(value)
    if rest >= 0.5 and value >= 0:
        return float(int(value) + 1)
    if rest <= -0.5 and value < 0:
        return float(int(value) - 1)
    return float(int(value))

@njit
def get_boundary_conditions(
        cell_dimensions: np.ndarray,
        particles_number: int,
        positions: np.ndarray,
):
    for i in range(particles_number):
        for j in range(3):
            if positions[i][j] >= cell_dimensions[j] / 2.0:
                positions[i][j] -= (
                        math_round(positions[i][j] / cell_dimensions[j])
                        * cell_dimensions[j]
                )
            if positions[i][j] < -cell_dimensions[j] / 2.0:
                positions[i][j] -= (
                        math_round(positions[i][j] / cell_dimensions[j])
                        * cell_dimensions[j]
                )
    return positions

In [ ]:
def calculate_rdf(positions):
    cell_dimensions = np.array([12.25, 12.25, 12.25])
    radii, rdf = OvitoProcessor(
        positions=positions, 
        cell_dimensions=cell_dimensions,
    ).get_rdf()
    return radii, rdf

In [ ]:
def mse(obs, exp):
    return np.sum((obs - exp) ** 2)

In [ ]:
def generate_random_state() -> None:
    particles_number = 1372
    cell_dimensions = 12.25 * np.ones(3)
    return (np.random.random((particles_number, 3)) - 0.5) * cell_dimensions

In [ ]:
CURRENT_DATA_NORMAL = '2021-09-24_prepared_1.3_normal'
PLOT_FILENAME_POSTFIX_NORMAL = 'prepared_1.3_normal'

PATH_TO_CURRENT_DATA_NORMAL = os.path.join(con.PATH_TO_DATA, CURRENT_DATA_NORMAL)
PATH_TO_CURRENT_DATA_NORMAL

In [ ]:
samples = np.load(os.path.join(PATH_TO_CURRENT_DATA_NORMAL, 'samples.npy'))
test_sample = np.load(os.path.join(PATH_TO_CURRENT_DATA_NORMAL, 'test_sample.npy'))
targets = np.load(os.path.join(PATH_TO_CURRENT_DATA_NORMAL, 'targets_pbc.npy'))
cooled_1 = np.load(os.path.join(PATH_TO_CURRENT_DATA_NORMAL, 'cooled_1.npy'))
cooled_2 = np.load(os.path.join(PATH_TO_CURRENT_DATA_NORMAL, 'cooled_2.npy'))

samples.shape, test_sample.shape, targets.shape, cooled_1.shape, cooled_2.shape

In [ ]:
i = 1
obs_rdf = samples[i]
some_positions = deepcopy(targets[i])
# some_positions = generate_random_state()
some_positions.shape

In [ ]:
radii, exp_rdf = calculate_rdf(some_positions)
size = obs_rdf.size
radii = radii[:size]
exp_rdf = exp_rdf[:size]
print(mse(obs=obs_rdf, exp=exp_rdf))
plt.plot(radii, exp_rdf, color='red')
plt.plot(radii, obs_rdf, color='black')
plt.show()

In [ ]:
def optimized_func(positions):
    collided_number = 0
    positions = positions.reshape(1372, 3)
    radii, exp_rdf = calculate_rdf(positions)
    size = obs_rdf.size
    radii = radii[:size]
    exp_rdf = exp_rdf[:size]
    if np.where(exp_rdf > 0)[0][0] < np.where(obs_rdf > 0)[0][0]:
        collided_number = exp_rdf[np.where(exp_rdf > 0)[0]].sum()
    result = mse(obs=obs_rdf, exp=exp_rdf)
    print(f'{result:8.3f}, {collided_number:8.3f}, {result + collided_number:8.3f}')
    return result, collided_number

In [ ]:
new_res = dual_annealing(
    lambda x: optimized_func(x)[0],
    x0=some_positions.reshape(1372*3),
    bounds=[(-6.125, 6.125) for _ in range(1372*3)],
    no_local_search=True,
)
new_res = dual_annealing(
    lambda x: optimized_func(x)[1],
    x0=new_res.x[0].reshape(1372*3),
    bounds=[(-6.125, 6.125) for _ in range(1372*3)],
    no_local_search=True,
)

In [ ]:
new_res = differential_evolution(
    lambda x: optimized_func(x)[0] + optimized_func(x)[1],
    bounds=[(-6.125, 6.125) for _ in range(1372 * 3)],
    popsize=2,
)

In [ ]:
radii, new_exp_rdf = calculate_rdf(np.reshape(new_res.x[0], (1372, 3)))
size = obs_rdf.size
radii = radii[:size]
new_exp_rdf = new_exp_rdf[:size]

In [ ]:
print(f'Initial MSE = {mse(obs=obs_rdf, exp=exp_rdf)}')
print(f'Current MSE = {mse(obs=obs_rdf, exp=new_exp_rdf)}')
print(f'Current MSE = {optimized_func(new_res.x)[0]}')

In [ ]:
# plt.plot(radii, obs_rdf, color='black')
plt.plot(radii, exp_rdf, color='red')
plt.plot(radii, new_exp_rdf, color='green')
plt.show()

In [ ]:
plt.plot(radii, exp_rdf - obs_rdf, color='red')
plt.plot(radii, new_exp_rdf - obs_rdf, color='green')
plt.show()